In [1]:
import os

os.chdir('../data')
os.getcwd()

'/Users/sagar/ms-iiit/is-music/data'

In [2]:
import json

import pandas as pd
from pydub import AudioSegment
from moviepy.editor import *

In [3]:
AUDIO_CLIPS_DATA_DIR = './yt8m-audio-clips'
VIDEO_CLIPS_DATA_DIR = './yt8m-video-clips'
INFO_FILENAME = './yt8m-info.jsonl'
LABELS_FILENAME = './yt8m-labels.jsonl'
DATASET_PATH = './yt8m-clips-dataset-info'
EXCLUDED_VIDS = './dataset-excluded-ids.txt'
train_splits = ['train1', 'train2', 'train3', 'train4', 'train5']

In [4]:
info_df = pd.read_json(INFO_FILENAME, lines=True)

print(info_df.info())

info_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30026 entries, 0 to 30025
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   vid               30026 non-null  object             
 1   title             29225 non-null  object             
 2   description       29225 non-null  object             
 3   publish_time      29225 non-null  datetime64[ns, UTC]
 4   channel_id        29225 non-null  object             
 5   channel_title     29225 non-null  object             
 6   tags              29225 non-null  object             
 7   view_count        29225 non-null  float64            
 8   like_count        29225 non-null  float64            
 9   comment_count     29225 non-null  float64            
 10  fav_count         29225 non-null  float64            
 11  duration          29225 non-null  object             
 12  definition        29225 non-null  object             
 13  l

,vid,title,description,publish_time,channel_id,channel_title,tags,view_count,like_count,comment_count,fav_count,duration,definition,licensed_content,content_rating,projection,dimension,caption
0,l2qBnK3Phks,Scorpions - WIND OF CHANGE - Nürnberg Arena 25...,The Scorpions live at the Arena in Nuremberg (...,2010-11-29 15:01:51+00:00,UCYckckJlBEc4OVaT6ds63Bw,Marco's Music Channel,"Scorpions (Musical Group),Wind Of Change,Hard ...",9640.0,23.0,0.0,0.0,5:24,hd,1.0,{},rectangular,2d,false
1,MakeMGirX6I,Stoyan Yankoulov,www.stoyankoulov.com\nSolo Tupan\n\n\nyankulov,2007-08-10 20:40:46+00:00,UChqjaIpaE8QRd1AJYLMB_fA,Stoyan Yankoulov,"World,Music,Ethno,Jazz,Folk,Drums,Percussion,E...",56723.0,232.0,16.0,0.0,2:43,sd,0.0,{},rectangular,2d,false
2,GXHQCGKux8k,Molotov - Puto - La Trastienda 27.10.10,Recital Precensal\n=),2010-10-30 15:19:03+00:00,UC1z_7LGKBe749qVfea9tOyQ,ChechuManson,MOV02982,8490.0,22.0,6.0,0.0,4:19,sd,0.0,{},rectangular,2d,false
3,cdOe_wq7_oU,Branden From Neon Trees Demos the Mike Lull 'J...,Neon Trees' Bassist Branden Campbell stopped b...,2014-08-12 20:45:48+00:00,UCI5tGbwiVHy4BsZUXcSWvwQ,Chicago Music Exchange,"cme,chicago music exchange,review,demo,Neon Tr...",43298.0,234.0,39.0,0.0,5:20,hd,1.0,{},rectangular,2d,false
4,61c9_avmN9A,"Chimpanzees feeling the Beat of the Drum "" LI...",Chimps at Chimp Haven jamming to the beat of t...,2008-08-29 08:27:04+00:00,UC9Y2dHQ4h6p9L7yVjQTdoPg,Ron Hardy,"Chimpanzee,Shreveport,Louisiana,chimp,haven,fu...",99811.0,1400.0,246.0,0.0,4:37,sd,0.0,{},rectangular,2d,false


In [5]:
labels_df = pd.read_json(LABELS_FILENAME, lines=True)

print(labels_df.info())

labels_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168559 entries, 0 to 168558
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   vid     168559 non-null  object
 1   labels  168559 non-null  object
dtypes: object(2)
memory usage: 2.6+ MB
None


,vid,labels
0,l2qBnK3Phks,"[Concert, Drummer]"
1,MakeMGirX6I,"[Musician, Drum kit, Drum, Drummer]"
2,GXHQCGKux8k,[Concert]
3,cdOe_wq7_oU,"[Musician, Guitar, String instrument, Electric..."
4,61c9_avmN9A,[Drum]


In [6]:
split = 'train'
clip_infos = list()
vid_infos = list()
for curr_split in train_splits:
    audio_info_filename = f'audio-clips-info-{curr_split}.jsonl'
    audio_info_filepath = os.path.join(AUDIO_CLIPS_DATA_DIR, audio_info_filename)
    video_info_filename = f'video-clips-info-{curr_split}.jsonl'
    video_info_filepath = os.path.join(VIDEO_CLIPS_DATA_DIR, video_info_filename)

    audio_info_df = pd.read_json(audio_info_filepath, lines=True)
    video_info_df = pd.read_json(video_info_filepath, lines=True)
    audio_vids = audio_info_df['vid'].tolist()
    video_vids = video_info_df['vid'].tolist()
    vids = list(set(audio_vids).intersection(set(video_vids)))
    
    for i, vid in enumerate(vids):
        if i > 0 and i % 100 == 0:
            print(curr_split, i)
        audio_clips_dir = os.path.join(AUDIO_CLIPS_DATA_DIR, curr_split, vid)
        video_clips_dir = os.path.join(VIDEO_CLIPS_DATA_DIR, curr_split, vid)
        audio_info = audio_info_df[audio_info_df['vid'] == vid].iloc[0]
        video_info = video_info_df[video_info_df['vid'] == vid].iloc[0]
        audio_clip_filenames = os.listdir(audio_clips_dir)
        video_clip_filenames = os.listdir(video_clips_dir)
        clip_nos = [int(fn[:-4].split('-')[-1]) for fn in audio_clip_filenames]
        v_clip_nos = [int(fn[:-4].split('-')[-1]) for fn in video_clip_filenames]
        vid_clip_infos = list()

        try:
            assert audio_info['n_clips'] == video_info['n_clips']
            assert audio_info['n_sampled_clips'] == video_info['n_sampled_clips']
            assert audio_info['n_sampled_clips'] == len(audio_clip_filenames)
            assert video_info['n_sampled_clips'] == len(video_clip_filenames)
            assert set(clip_nos) == set(v_clip_nos)
        except AssertionError:
            with open(EXCLUDED_VIDS, 'a') as f:
                f.write(vid + f',{split}\n')
            continue

        audio_sampled_dur = 0
        video_sampled_dur = 0
        for c in clip_nos:
            audio_filename = f'{vid}-audio-{c}.mp3'
            audio_filepath = os.path.join(audio_clips_dir, audio_filename)
            audio_clip = AudioSegment.from_file(audio_filepath)
            audio_clip_dur = len(audio_clip)/1000.0
            audio_sampled_dur += audio_clip_dur
            
            video_filename = f'{vid}-video-{c}.mp4'
            video_filepath = os.path.join(video_clips_dir, video_filename)
            video_clip = VideoFileClip(video_filepath)
            video_clip_dur = video_clip.duration
            video_sampled_dur += video_clip_dur

            try:
                assert abs(audio_clip_dur - video_clip_dur) < 1.0
            except AssertionError:
                with open(EXCLUDED_VIDS, 'a') as f:
                    f.write(vid + f',{split}\n')
                continue

            vid_clip_infos.append({
                'vid': vid,
                'clip_no': c,
                'audio_clip_name': audio_filename,
                'audio_clip_dur': audio_clip_dur,
                'video_clip_name': video_filename,
                'video_clip_dur': video_clip_dur
            })
        
        try:
            assert abs(audio_info['sampled_dur'] - audio_sampled_dur) < 4.0
            assert abs(video_info['sampled_dur'] - video_sampled_dur) < 4.0
        except AssertionError:
            with open(EXCLUDED_VIDS, 'a') as f:
                f.write(vid + f',{split}\n')
            continue

        clip_infos.extend(vid_clip_infos)
        info_row = info_df[info_df['vid'] == vid].iloc[0]
        vid_infos.append({
            'vid': vid,
            'n_clips': int(audio_info['n_clips']),
            'n_sampled_clips': int(audio_info['n_sampled_clips']),
            'audio_dur': float(audio_info['duration_sec']),
            'sampled_audio_dur': audio_sampled_dur,
            'video_dur': float(video_info['duration_sec']),
            'sampled_video_dur': video_sampled_dur,
            'split': curr_split,
            'labels': labels_df[labels_df['vid'] == vid].iloc[0]['labels'],
            'title': info_row['title'],
            'description': info_row['description'],
            'tags': info_row['tags']
        })

split_dataset_path = os.path.join(DATASET_PATH, split)
os.makedirs(split_dataset_path, exist_ok=True)
with open(os.path.join(split_dataset_path, 'clip-info.jsonl'), 'w') as f:
    f.write('\n'.join([json.dumps(c) for c in clip_infos]))
with open(os.path.join(split_dataset_path, 'video-info.jsonl'), 'w') as f:
    f.write('\n'.join([json.dumps(v) for v in vid_infos]))

len(clip_infos), len(vid_infos)

train1 100
train1 200
train1 300
train1 400
train1 500
train1 600
train1 700
train1 800
train1 900
train2 100
train2 200
train2 300
train2 400
train2 500
train2 600
train2 700
train2 800
train2 900
train3 100
train3 200
train3 300
train3 400
train3 500
train3 600
train3 700
train3 800
train3 900
train4 100
train4 200
train4 300
train4 400
train4 500
train4 600
train4 700
train4 800
train4 900
train5 100
train5 200
train5 300
train5 400
train5 500
train5 600
train5 700
train5 800
train5 900
train5 1000


(58847, 4831)

In [8]:
split = 'dev'
curr_split = split
clip_infos = list()
vid_infos = list()

audio_info_filename = f'audio-clips-info-{curr_split}.jsonl'
audio_info_filepath = os.path.join(AUDIO_CLIPS_DATA_DIR, audio_info_filename)
video_info_filename = f'video-clips-info-{curr_split}.jsonl'
video_info_filepath = os.path.join(VIDEO_CLIPS_DATA_DIR, video_info_filename)

audio_info_df = pd.read_json(audio_info_filepath, lines=True)
video_info_df = pd.read_json(video_info_filepath, lines=True)
audio_vids = audio_info_df['vid'].tolist()
video_vids = video_info_df['vid'].tolist()
vids = list(set(audio_vids).intersection(set(video_vids)))

for i, vid in enumerate(vids):
    if i > 0 and i % 100 == 0:
        print(i)
    audio_clips_dir = os.path.join(AUDIO_CLIPS_DATA_DIR, curr_split, vid)
    video_clips_dir = os.path.join(VIDEO_CLIPS_DATA_DIR, curr_split, vid)
    audio_info = audio_info_df[audio_info_df['vid'] == vid].iloc[0]
    video_info = video_info_df[video_info_df['vid'] == vid].iloc[0]
    audio_clip_filenames = os.listdir(audio_clips_dir)
    video_clip_filenames = os.listdir(video_clips_dir)
    clip_nos = [int(fn[:-4].split('-')[-1]) for fn in audio_clip_filenames]
    v_clip_nos = [int(fn[:-4].split('-')[-1]) for fn in video_clip_filenames]
    vid_clip_infos = list()

    try:
        assert audio_info['n_clips'] == video_info['n_clips']
        assert set(clip_nos) == set(v_clip_nos)
    except AssertionError:
        with open(EXCLUDED_VIDS, 'a') as f:
            f.write(vid + f',{split}\n')
        continue

    audio_sampled_dur = 0
    video_sampled_dur = 0
    for c in clip_nos:
        audio_filename = f'{vid}-audio-{c}.mp3'
        audio_filepath = os.path.join(audio_clips_dir, audio_filename)
        audio_clip = AudioSegment.from_file(audio_filepath)
        audio_clip_dur = len(audio_clip)/1000.0
        audio_sampled_dur += audio_clip_dur
        
        video_filename = f'{vid}-video-{c}.mp4'
        video_filepath = os.path.join(video_clips_dir, video_filename)
        video_clip = VideoFileClip(video_filepath)
        video_clip_dur = video_clip.duration
        video_sampled_dur += video_clip_dur

        try:
            assert abs(audio_clip_dur - video_clip_dur) < 1.0
        except AssertionError:
            with open(EXCLUDED_VIDS, 'a') as f:
                f.write(vid + f',{split}\n')
            continue

        clip_infos.append({
            'vid': vid,
            'clip_no': c,
            'audio_clip_name': audio_filename,
            'audio_clip_dur': audio_clip_dur,
            'video_clip_name': video_filename,
            'video_clip_dur': video_clip_dur
        })
    
    try:
        assert abs(audio_info['duration_sec'] - audio_sampled_dur) < 4.0
        assert abs(video_info['duration_sec'] - video_sampled_dur) < 4.0
    except AssertionError:
        with open(EXCLUDED_VIDS, 'a') as f:
            f.write(vid + f',{split}\n')
        continue
    
    clip_infos.extend(vid_clip_infos)
    info_row = info_df[info_df['vid'] == vid].iloc[0]
    vid_infos.append({
        'vid': vid,
        'n_clips': int(audio_info['n_clips']),
        'audio_dur': float(audio_info['duration_sec']),
        'video_dur': float(video_info['duration_sec']),
        'split': curr_split,
        'labels': labels_df[labels_df['vid'] == vid].iloc[0]['labels'],
        'title': info_row['title'],
        'description': info_row['description'],
        'tags': info_row['tags']
    })

split_dataset_path = os.path.join(DATASET_PATH, split)
os.makedirs(split_dataset_path, exist_ok=True)
with open(os.path.join(split_dataset_path, 'clip-info.jsonl'), 'w') as f:
    f.write('\n'.join([json.dumps(c) for c in clip_infos]))
with open(os.path.join(split_dataset_path, 'video-info.jsonl'), 'w') as f:
    f.write('\n'.join([json.dumps(v) for v in vid_infos]))

len(clip_infos), len(vid_infos)

100
200
300
400
500


(15961, 499)

In [9]:
split = 'test'
curr_split = split
clip_infos = list()
vid_infos = list()

audio_info_filename = f'audio-clips-info-{curr_split}.jsonl'
audio_info_filepath = os.path.join(AUDIO_CLIPS_DATA_DIR, audio_info_filename)
video_info_filename = f'video-clips-info-{curr_split}.jsonl'
video_info_filepath = os.path.join(VIDEO_CLIPS_DATA_DIR, video_info_filename)

audio_info_df = pd.read_json(audio_info_filepath, lines=True)
video_info_df = pd.read_json(video_info_filepath, lines=True)
audio_vids = audio_info_df['vid'].tolist()
video_vids = video_info_df['vid'].tolist()
vids = list(set(audio_vids).intersection(set(video_vids)))

for i, vid in enumerate(vids):
    if i > 0 and i % 100 == 0:
        print(i)
    audio_clips_dir = os.path.join(AUDIO_CLIPS_DATA_DIR, curr_split, vid)
    video_clips_dir = os.path.join(VIDEO_CLIPS_DATA_DIR, curr_split, vid)
    audio_info = audio_info_df[audio_info_df['vid'] == vid].iloc[0]
    video_info = video_info_df[video_info_df['vid'] == vid].iloc[0]
    audio_clip_filenames = os.listdir(audio_clips_dir)
    video_clip_filenames = os.listdir(video_clips_dir)
    clip_nos = [int(fn[:-4].split('-')[-1]) for fn in audio_clip_filenames]
    v_clip_nos = [int(fn[:-4].split('-')[-1]) for fn in video_clip_filenames]
    vid_clip_infos = list()

    try:
        assert audio_info['n_clips'] == video_info['n_clips']
        assert set(clip_nos) == set(v_clip_nos)
    except AssertionError:
        with open(EXCLUDED_VIDS, 'a') as f:
            f.write(vid + f',{split}\n')
        continue

    audio_sampled_dur = 0
    video_sampled_dur = 0
    for c in clip_nos:
        audio_filename = f'{vid}-audio-{c}.mp3'
        audio_filepath = os.path.join(audio_clips_dir, audio_filename)
        audio_clip = AudioSegment.from_file(audio_filepath)
        audio_clip_dur = len(audio_clip)/1000.0
        audio_sampled_dur += audio_clip_dur
        
        video_filename = f'{vid}-video-{c}.mp4'
        video_filepath = os.path.join(video_clips_dir, video_filename)
        video_clip = VideoFileClip(video_filepath)
        video_clip_dur = video_clip.duration
        video_sampled_dur += video_clip_dur

        try:
            assert abs(audio_clip_dur - video_clip_dur) < 1.0
        except:
            with open(EXCLUDED_VIDS, 'a') as f:
                f.write(vid + f',{split}\n')
            continue

        vid_clip_infos.append({
            'vid': vid,
            'clip_no': c,
            'audio_clip_name': audio_filename,
            'audio_clip_dur': audio_clip_dur,
            'video_clip_name': video_filename,
            'video_clip_dur': video_clip_dur
        })
    try:
        assert abs(audio_info['duration_sec'] - audio_sampled_dur) < 4.0
        assert abs(video_info['duration_sec'] - video_sampled_dur) < 4.0
    except AssertionError:
        with open(EXCLUDED_VIDS, 'a') as f:
            f.write(vid + f',{split}\n')
        continue
    
    clip_infos.extend(vid_clip_infos)
    info_row = info_df[info_df['vid'] == vid].iloc[0]
    vid_infos.append({
        'vid': vid,
        'n_clips': int(audio_info['n_clips']),
        'audio_dur': float(audio_info['duration_sec']),
        'video_dur': float(video_info['duration_sec']),
        'split': curr_split,
        'labels': labels_df[labels_df['vid'] == vid].iloc[0]['labels'],
        'title': info_row['title'],
        'description': info_row['description'],
        'tags': info_row['tags']
    })

split_dataset_path = os.path.join(DATASET_PATH, split)
os.makedirs(split_dataset_path, exist_ok=True)
with open(os.path.join(split_dataset_path, 'clip-info.jsonl'), 'w') as f:
    f.write('\n'.join([json.dumps(c) for c in clip_infos]))
with open(os.path.join(split_dataset_path, 'video-info.jsonl'), 'w') as f:
    f.write('\n'.join([json.dumps(v) for v in vid_infos]))

len(clip_infos), len(vid_infos)

100
200
300
400


(14806, 461)